In [1]:
import datasets
import transformers
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
import torch

import os

from performanceBenchmark import PerformanceBenchmark
from dotenv import load_dotenv
load_dotenv(verbose=True)
api_token = os.getenv('HF_DATASET_API_TOKEN')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
dataset = datasets.load_dataset('metamong1/summarization_paper', use_auth_token=api_token)

Reusing dataset paper_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/1.4.0/24bb09528ebb04fdc6aafb6e110202e52fbb818c0f204839bc833d8ce1e86a5f)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
TEST_DATASET_SIZE = 10

test_dataset = dataset['validation'].shuffle().filter(lambda x: len(x['text'])< 500).select(range(TEST_DATASET_SIZE))
# test_dataset = dataset['train'].shuffle().filter(lambda x: len(x['text']) < 500).select(range(TEST_DATASET_SIZE))

  0%|          | 0/19 [00:00<?, ?ba/s]

In [4]:
len(test_dataset)

10

In [5]:
test_dataset['text']

['본 연구는 거버넌스에 대한 다의적 해석에 근거하여 지방거버넌스의 체제 능력을 구성하는 요소 및 성과 측정의 기준에 대한 기존 연구의 성과와 한계를 밝히는 데 연구의 일차적인 목적을 두고 있다. 아울러 우리나라 지방정치 및 지방자치의 현실 속에서 그와 같은 체제 능력에 대한 분석이 어떻게 이루어지는 것이 바람직 한 지를 살펴보면서 민주주의적 규범의 토대 위에 거버넌스 발전의 이상형(ideal type)을 구축해 보는 데도 연구의 주된 목적을 두고 있다. 우리나라 지방정치의 구조적 특성은 시민사회, 언론, 기업 등 민간 혹은 정치사회 영역의 거버넌스 행위자들의 위상 약화로 인해 강시장형의 계층제적 성격이 강한 지방거버넌스 구조가 고착되고 있으며, 그러한 경향은 지역주의가 발호하는 지역일수록 더 심하게 나타난다. 이와 같은 우리나라 지방정치의 특성은 정치사회의 주요행위자인 정당의 역할과 기능의 부재에 기인한 바 크다.',
 '본 연구는 리만의 음악미학 문헌에 나타난 서술 경향을 당대의 주요 관련 저서들과 연계하여 비평적 관점에서 살펴보았다. ‘음악미학’이라는 제목을 사용한 대표적 초기 문헌인 리만의 저술은 19세기의 관련 논의들을 종합하고 정리하면서 음악에 대한 형이상학적 이해와 자연과학적 이해를 통합하려는 시도를 보여준다. 이를 통해 리만은 당대의 음악미학적 논의들을 음악이론적 근거 도출이라는 흐름에 합류시키고 있다. 비슷한 시기에 출판된 엥겔 및 발라쉑의 음악미학 문헌 역시 기본적으로 그러한 내용을 담고 있으나, 리만의 문헌에서 그것들이 보다 긴밀한 상호 관련성을 지닌 형태로 재구성되어 나타난다. 리만의 이러한 시도는 동시대의 쟁점들을 포괄하고 있는데, 헤겔 미학과 한슬릭의 형식주의를 아우르고 있다. 그럼에도 불구하고 리만의 견해는 여전히 상충되는 미학적 사항들을 해결해야 하는 과제를 안고 있는 것으로 여겨진다.',
 '선체에 존재하는 균열의 전파거동을 해석하기 위해서는 응력강도계수의 추정이 그 전제조건이 되지만 현재까지 이러한 복잡 구조물에서 응력강도계수를

In [6]:
check_point = 'pruned_last_layer-finetuned-paper-sample-size-1000-pruned/checkpoint-1000'

In [7]:
tokenizer = AutoTokenizer.from_pretrained(check_point)

In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained(check_point).to(device)

/opt/conda/envs/final/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [9]:
summerizer = pipeline(
    'summarization', 
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
    )

In [10]:
summerizer(test_dataset['text'])

[{'summary_text': '거버넌스 체제 능력 평가의 한 양상 - 부산경남지역을 중심으로- A'},
 {'summary_text': '리만의의 음악미학 : 19세기 음악 저서를 중심으로 ( 엥겔 및'},
 {'summary_text': '복잡전파거동 해석을 위한 응력강도계수 계산에 관한'},
 {'summary_text': '한국어 명사 단어의 연상 빈도와 정의에 관한 연구-단어 당 100'},
 {'summary_text': '방과 후 태권도 교육 참여 초등학생들의 체력, 품성, 기본심리욕구 차이 연구'},
 {'summary_text': '비버 챌린지 2017의 초등 컴퓨터과학 적용 결과 분석 및 초등학교 정보 교육과정'},
 {'summary_text': '만주국의 건국 이데올로기로서 황도주의와 민족협화 - 재만 일본인을'},
 {'summary_text': '아동상담자의 수퍼비젼 경험과 자기분석 경험에 따른 대리외상 경험의'},
 {'summary_text': '디지털 증거에 대한 효과적인 파일 카빙 기법 연구 - 레코드 파일 카빙 기'},
 {'summary_text': '중장년층의 취업 및 창업을 지원하기 위한 한국폴리텍대학 베이비부'}]

In [11]:
performance_benchmark = PerformanceBenchmark(summerizer, test_dataset, tokenizer, 'baseline')

In [12]:
performance_benchmark.compute_rouge()

====ROUGE score====
{'rouge1': 43.6774, 'rouge2': 28.8295, 'rougeL': 39.9166, 'rougeLsum': 39.4778}


{'rouge1': 43.6774, 'rouge2': 28.8295, 'rougeL': 39.9166, 'rougeLsum': 39.4778}

In [13]:
performance_benchmark.compute_size()

Model size (MB) = 472.6946954727173


{'size_mb': 472.6946954727173}

In [14]:
performance_benchmark.compute_time()

/opt/conda/envs/final/lib/python3.8/site-packages/transformers/pipelines/base.py:899: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Average time took(ms) 1.2e+02 +\- 1.56


{'time_avg_ms': 117.41239743307233, 'time_std_ms': 1.5579766904366878}